## Web scraping of RouteScanner

A first notebook to web scrape Route Scanner.

In [1]:
# Quick and dirty hack because my PyCharm interpreter is not playing nice
ewouts_broken_PyCharm = True
if ewouts_broken_PyCharm:
    import sys
    sys.path.append('C:\\Users\\Ewout\\Documents\\python_venv\\Py310')
    sys.path.append('C:\\Users\\Ewout\\Documents\\python_venv\\Py310\\lib\\site-packages')

### Direct connections

In this part a dataframe is created with all direct connections between sea terminals in South America and The Netherlands/Belgium.

Data: https://www.routescanner.com/services/direct-connections

**TODO:** Make sure more than 20 routes per webpage can be scraped (the results on page 2 and onwards)

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import os
import itertools
import pandas as pd
import pickle

#### Webscraping

In [3]:
# Define origin and destination countries
origin = ["BR", "CO", "VE", "SR", "CW", "GY", "GF", "UY", "AR", "CL", "PE", "EC"]
destination = ["NL", "BE"]

# Make list with all combinations
od_list = list(itertools.product(origin, destination))
print(od_list)

# Create URL list to scrape
urls = [f"https://www.routescanner.com/services/direct-connections/results?destinationCountries={d}&originCountries={o}&search=advanced" for o, d in od_list]

[('BR', 'NL'), ('BR', 'BE'), ('CO', 'NL'), ('CO', 'BE'), ('VE', 'NL'), ('VE', 'BE'), ('SR', 'NL'), ('SR', 'BE'), ('CW', 'NL'), ('CW', 'BE'), ('GY', 'NL'), ('GY', 'BE'), ('GF', 'NL'), ('GF', 'BE'), ('UY', 'NL'), ('UY', 'BE'), ('AR', 'NL'), ('AR', 'BE'), ('CL', 'NL'), ('CL', 'BE'), ('PE', 'NL'), ('PE', 'BE'), ('EC', 'NL'), ('EC', 'BE')]


In [4]:
def get_webpage(url, headless=False):
    # Instantiate options
    opts = Options()
    opts.binary_location = "C:\Program Files\Google\Chrome\Application\chrome.exe"
    if headless:
        opts.headless = True

    # Set the location of the webdriver
    chrome_driver = os.getcwd() + "/drivers/chromedriver.exe"

    # Instantiate a webdriver
    driver = webdriver.Chrome(options=opts, executable_path=chrome_driver)

    # Load the HTML page
    driver.get(url)

    # Accept cookies
    driver.implicitly_wait(2)
    driver.find_element(By.CLASS_NAME,"acceptButton__P2szu").click()

    # Function to get soup from the drivers
    def get_soup():
        # Wait untill route data is loaded
        try:
            elem = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, 'card__hoI9D')))
        except:
            print('No connections found on this route')

        # Parse processed webpage with BeautifulSoup and save in list
        return BeautifulSoup(driver.page_source)

    # Save the first soup in a new list
    soups = [get_soup()]

    # Calculate number of results
    n_results = int(soups[0].find("h6", class_="totalResults__mTaYp").text.split(' ')[0])

    # Get results from other pages
    if n_results > 20 and recursive:
        # Calculate number of extra pages
        extra_pages = (n_results-1) // 20
        # Calculate data for each extra page
        for n in range(2, extra_pages+2):
            # Go to extra page, get soup and append to soups list
            driver.get(f"{url}&page={n}")
            soups.append(get_soup())

    # Close the browser and return soups list and n_results
    driver.close()
    return soups, n_results

In [5]:
# A test run with a single webpage
soups1, n1 = get_webpage(urls[0])
print(f"{n1} results across {len(soups1)} pages")
soups1[0]

C:\Users\Ewout\AppData\Local\Temp\ipykernel_7416\3882681091.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=opts, executable_path=chrome_driver)


31 results across 2 pages


<html lang="en" translate="no" xmlns:fb="http://ogp.me/ns/fb#"><head><meta content="width=device-width,initial-scale=1,shrink-to-fit=no" name="viewport"/><meta content="notranslate" name="google"/><meta charset="utf-8"/><title>Explore direct container connections from one place to another</title><meta content="Find the best door-to-door shipping routes by sea and inland. Compare options on lead time and CO2 emissions." name="description"/><meta content="routescanner, container shipping routes, maritime route, routeplanner, shortsea connections, deepsea connections, rail connections, barge connections" name="keywords"/><meta content="For the best container shipping route comparison, use Routescanner to find the best connection by deepsea, feeder, rail, barge and truck." itemprop="name"/><meta content="Routescanner – Make better container shipping decisions" property="og:title"/><meta content="Routescanner – Plan your door-to-door container shipping route" property="og:site_name"/><meta 

#### Extract data from HTML

In [6]:
# Returns route data for list of routes
def get_route_data(soups):
    # Create empty list for the route data
    route_data = []

    # Iterate through soup pages
    for soup in soups:
        # Create a list with all route cards
        routes = soup.find_all("div", class_="card__hoI9D")

        # For each route, find the company, origin, destination, service code, duration and if known frequency
        for route in routes:
            c = route.find("h6").text
            o, d = route.find("small", class_="locodes__tdIbs").text.split(' - ')
            s = route.find("p", class_="serviceCode__igooW").text
            t = route.find("div", class_="times__xgwdu").find_all("div")
            t = [i.text for i in t]
            route_data.append([c, o, d, s, *t])
    return route_data

In [7]:
# Get route data sample
data1 = get_route_data(soups1)

# Define dict keys / dataframe columns
columns = ["Company", "Origin", "Destination", "Service Code", "Duration", "Frequency"]

# Create dataframe from the data sample
df1 = pd.DataFrame(data1, columns=columns)
print(f"Created a DataFrame with {df1.index.size} entries")
df1.head()

Created a DataFrame with 31 entries


,Company,Origin,Destination,Service Code,Duration,Frequency
0,MSC,BRRIG,NLRTM,NWC TO SAEC - STRING I,25 days,None
1,Hamburg Sud,BRPNG,NLRTM,SOUTH AMERICA EAST COAST -- NORTH EUROPE SLING...,21 days,None
2,MSC,BRPNG,NLRTM,NWC TO SAEC - STRING I,21 days,1 time per week
3,Hapag-Lloyd,BRPNG,NLRTM,ECX-EUROPE EAST COAST EXPRESS,20 - 21 days,1 time per week
4,CMA CGM,BRSSZ,NLRTM,SOUTH AMERICA FR. NORTH EUR. PLATE SLING,19 days,None


#### Run on all routes

In [8]:
# The data can be loaded from a pickle, so the script doesn't need to re-run every time.
# Set use_pickle1 to False if you want to scrape all the data again.
use_pickle1 = True

if use_pickle1:
    route_df = pd.read_pickle("pickles/routes_between_ports.pickle")

else:
    # Empty list for data and len
    route_data = []
    combinations = len(urls)

    for n, url in enumerate(urls):
        # Scrape the webpage
        soups, _ = get_webpage(url, headless=True)
        # Get all the route data, process them and add
        new_route_data = get_route_data(soups)
        route_data = route_data + new_route_data
        # Nice print message
        print(f"Done with {n}/{combinations} webpages, {len(new_route_data)} new routes found (total {len(route_data)})")

    # Create a dataframe from all the route data
    route_df = pd.DataFrame(route_data, columns=columns)
route_df.head()

C:\Users\Ewout\AppData\Local\Temp\ipykernel_7416\3882681091.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=opts, executable_path=chrome_driver)


Done with 0/24 webpages, 31 new routes found (total 31)
Done with 1/24 webpages, 14 new routes found (total 45)
Done with 2/24 webpages, 15 new routes found (total 60)
Done with 3/24 webpages, 18 new routes found (total 78)
No connections found on this route
Done with 4/24 webpages, 0 new routes found (total 78)
No connections found on this route
Done with 5/24 webpages, 0 new routes found (total 78)
Done with 6/24 webpages, 1 new routes found (total 79)
No connections found on this route
Done with 7/24 webpages, 0 new routes found (total 79)
Done with 8/24 webpages, 3 new routes found (total 82)
No connections found on this route
Done with 9/24 webpages, 0 new routes found (total 82)
No connections found on this route
Done with 10/24 webpages, 0 new routes found (total 82)
No connections found on this route
Done with 11/24 webpages, 0 new routes found (total 82)
Done with 12/24 webpages, 1 new routes found (total 83)
No connections found on this route
Done with 13/24 webpages, 0 new r

,Company,Origin,Destination,Service Code,Duration,Frequency
0,MSC,BRRIG,NLRTM,NWC TO SAEC - STRING I,25 days,None
1,Hamburg Sud,BRPNG,NLRTM,SOUTH AMERICA EAST COAST -- NORTH EUROPE SLING...,21 days,None
2,MSC,BRPNG,NLRTM,NWC TO SAEC - STRING I,21 days,1 time per week
3,Hapag-Lloyd,BRPNG,NLRTM,ECX-EUROPE EAST COAST EXPRESS,20 - 21 days,1 time per week
4,CMA CGM,BRSSZ,NLRTM,SOUTH AMERICA FR. NORTH EUR. PLATE SLING,19 days,None


In [9]:
# Save as Pickle and CSV
route_df.to_pickle("pickles/routes_between_ports.pickle")
route_df.to_csv("data/routes_between_ports.csv")